In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

2025-03-04 00:54:03.680191: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load and preprocess data (training, validation, and test sets)
train_dir = "/Users/nasrinjahan/Desktop/Indivisual Project/Project_NK/machineLearning_PlantDiseaseDetectionSystem/Plant_Diseases_Dataset/train"
valid_dir = "/Users/nasrinjahan/Desktop/Indivisual Project/Project_NK/machineLearning_PlantDiseaseDetectionSystem/Plant_Diseases_Dataset/valid"
test_dir = "/Users/nasrinjahan/Desktop/Indivisual Project/Project_NK/machineLearning_PlantDiseaseDetectionSystem/Plant_Diseases_Dataset/test"

In [5]:
# Image size and batch size
img_size = (128, 128)
batch_size = 32

In [7]:
# Image preprocessing and augmentation
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [9]:
train_generator = datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='sparse')
test_generator = datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='sparse', shuffle=False)
valid_generator = datagen.flow_from_directory(valid_dir, target_size=img_size, batch_size=batch_size, class_mode='sparse', shuffle=False)

Found 70295 images belonging to 38 classes.
Found 33 images belonging to 1 classes.
Found 17572 images belonging to 38 classes.


In [11]:
# Extract class names
class_names = list(train_generator.class_indices.keys())

In [13]:
# Load a pre-trained CNN (ResNet50) for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze layers

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [15]:
# Add a global pooling layer
model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

In [17]:
# Extract features
X_train = model.predict(train_generator)
y_train = train_generator.classes

X_test = model.predict(test_generator)
y_test = test_generator.classes

X_valid = model.predict(valid_generator)
y_valid = valid_generator.classes

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1331s 605ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step 
550/550 ━━━━━━━━━━━━━━━━━━━━ 434s 789ms/step


In [ ]:
# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Predictions
y_pred = rf_model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)

In [ ]:
# Classification Report
print("Test Accuracy:", test_acc)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=False, cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Feature Importance Visualization
feature_importances = rf_model.feature_importances_
plt.figure(figsize=(12, 6))
plt.bar(range(len(feature_importances)), feature_importances)
plt.xlabel("Feature Index")
plt.ylabel("Importance Score")
plt.title("Feature Importances from Random Forest")
plt.show()